In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [3]:
dataset=pd.read_csv("dataset.csv")
print(dataset.shape)
data_size=len(dataset)
print (data_size)

(16602, 3)
16602


In [4]:
import re
import nltk
from urllib.parse import urlparse
#nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

corpus=[]

for i in range(0,data_size):
    tweet=dataset['text'][i]
    
    tweet = re.sub('[^a-zA-Z]', ' ', dataset['text'][i])
    tweet=tweet.lower()
    tweet=tweet.split()
    ps=PorterStemmer()
    tweet=[ps.stem(word) for word in tweet if word not in set(stopwords.words('english')) and len(word)>2]
    tweet=' '.join(tweet)
    corpus.append(tweet)
    
    

In [5]:
words=[]
for i in corpus:
    for word in i:
        words.append(i)
print (len(words))
print ((corpus[0]))

1050357
nepal nsent whatsapp http cla kaa


In [6]:
import collections
counter=collections.Counter(words)
common=counter.most_common(50)
common_words=[]
for i in range(0,50):
    wo=common[i][0].split()
    for w in wo:
        common_words.append(w)

In [7]:
print (len(common_words))
common_words=list(set(common_words))
print (len(common_words))
print (common_words)

585
359
['even', 'relief', 'alamgirizvi', 'loss', 'pvindian', 'properti', 'part', 'appeal', 'mimarathinew', 'mani', 'bengal', 'ijaideep', 'drunkvinodmehta', 'run', 'action', 'understandin', 'assam', 'say', 'deepli', 'simmilar', 'bihar', 'nriot', 'guindi', 'spoonia', 'miss', 'depict', 'etc', 'aftr', 'crack', 'airlift', 'sagarikaghos', 'read', 'armi', 'fallen', 'sleepless', 'ravindraambekar', 'love', 'dmk', 'moron', 'shahidkapoor', 'sikhism', 'affect', 'maharaj', 'hit', 'facebook', 'come', 'launch', 'tulsidasbhoit', 'anandmahindra', 'busi', 'bless', 'nepalearthquak', 'realli', 'leader', 'neighbour', 'hinder', 'year', 'confirm', 'famili', 'cpi', 'respond', 'locat', 'allow', 'unit', 'donat', 'sabha', 'eastern', 'aftershock', 'east', 'drbsinsa', 'bainjal', 'occurrenc', 'work', 'http', 'cow', 'daili', 'forward', 'rajasthan', 'friend', 'ngolden', 'next', 'right', 'break', 'bike', 'thi', 'johngreen', 'indiawithnep', 'highest', 'instanc', 'mp', 'meet', 'googleindia', 'manishtamak', 'philip', 'n

In [8]:
imp_words=['medic','need','give','relief','fund','food','donat','aid','water','meal','send','offer','financ','blood']


In [9]:
common_words=[i for i in common_words if i not in imp_words]
print (len(common_words))

346


In [10]:
final_corpus=[]
for i in corpus:
    final_corpus.append(' '.join([w for w in i.split() if w not in common_words]))

In [11]:
words=[]
for i in final_corpus:
    for word in i:
        words.append(i)
print (len(words))

634402


In [12]:
Y=dataset.iloc[:,2].values
Y=list(Y)
print (len(final_corpus))
seen = set()
result = []
removed_indices=[]
for i in range(0,data_size):
    if final_corpus[i] not in seen:
        seen.add(final_corpus[i])
        result.append(final_corpus[i])
    else:
        removed_indices.insert(0,i)
final_corpus=result
for i in removed_indices:
    del(Y[i])
print (len(final_corpus))
print (len(Y))

16602
14257
14257


In [13]:
count=0
for i in Y:
    if i==2 or i==1:
        count+=1
print (count)


875


In [14]:
delIndex=[]
l=len(Y)
c=0
for i in range(0,l):
    if(Y[i]==0):
        if(c==count):
            delIndex=[i]+delIndex
        else:
            c+=1
for i in delIndex:
    final_corpus.pop(i)
    Y.pop(i)
print(c)

875


In [15]:
print(len(final_corpus))
print(len(Y))

1750
1750


In [14]:
testset=pd.read_csv("TestDataset.csv")
print(testset.shape)
data_size=len(testset)
print (data_size)

(46921, 2)
46921


In [15]:
import re
import nltk
from urllib.parse import urlparse
#nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

testdata=[]

for i in range(0,data_size):
    tweet=testset['text'][i]
    
    tweet = re.sub('[^a-zA-Z]', ' ', testset['text'][i])
    tweet=tweet.lower()
    tweet=tweet.split()
    ps=PorterStemmer()
    tweet=[ps.stem(word) for word in tweet if word not in set(stopwords.words('english')) and len(word)>2]
    tweet=' '.join(tweet)
    testdata.append(tweet)
    
    

In [19]:
from sklearn.pipeline import Pipeline,FeatureUnion
from sklearn import feature_selection
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn import tree
from sklearn.feature_extraction.text import TfidfTransformer,TfidfVectorizer
from sklearn.externals import joblib

LinearSVC1 = LinearSVC(tol=1e-4,  C = 0.10000000000000001)
f5 = feature_selection.RFE(estimator=LinearSVC1, n_features_to_select=1000, step=1)

pipeline=Pipeline([
    #('vectorizer',  CountVectorizer()),
    ('features', FeatureUnion([
       ('tfidf', TfidfVectorizer(ngram_range=(1, 2), max_features= 6000))])), 
    ('rfe_feature_selection', f5),
    #('classifier', LinearSVC1),
    #])
    ('tfidf_transformer',  TfidfTransformer()),
    ('classifier',  tree.DecisionTreeClassifier()) ])
pipeline.fit(final_corpus,Y)
joblib.dump(pipeline,"model.pkl")

['model.pkl',
 'model.pkl_01.npy',
 'model.pkl_02.npy',
 'model.pkl_03.npy',
 'model.pkl_04.npy',
 'model.pkl_05.npy',
 'model.pkl_06.npy',
 'model.pkl_07.npy',
 'model.pkl_08.npy',
 'model.pkl_09.npy',
 'model.pkl_10.npy',
 'model.pkl_11.npy',
 'model.pkl_12.npy',
 'model.pkl_13.npy']

In [16]:
from sklearn.externals import joblib
pipeline=joblib.load("model.pkl")
y_pred=pipeline.predict(testdata)


In [17]:
y_prob=pipeline.predict_proba(testdata)

In [18]:
print (y_prob[0][2])

0.0357142857143


In [33]:
support=pipeline.named_steps['rfe_feature_selection'].support_
feature_names = pipeline.named_steps['features'].get_feature_names()

In [12]:
import graphviz
import pydotplus
from sklearn import tree
from sklearn.externals.six import StringIO
dot_data = StringIO()
tree.export_graphviz(pipeline.named_steps['classifier'],out_file=dot_data,feature_names=feature_names,class_names=['0','1','2'],filled=True, rounded=True,special_characters=True) 
pydotplus.graph_from_dot_data(dot_data.getvalue()).write_png("finaltree.png")

True

In [19]:
print (len(y_pred))

46921


In [20]:
count0=0
count1=0
count2=0
for i in y_pred:
    if i==0:
        count0+=1
    elif i==1:
        count1+=1
    elif i==2:
        count2+=1
print (count0)
print (count1)
print (count2)

44910
397
1614


In [21]:
from operator import itemgetter
output=[]
out_need=[]
out_avail=[]
for i in range(0,len(y_pred)):
    output.append([testset['id'][i],y_pred[i],y_prob[i][y_pred[i]]])
    if y_pred[i]==1:
        out_need.append([testset['id'][i],y_pred[i],y_prob[i][y_pred[i]]])
    if y_pred[i]==2:
        out_avail.append([testset['id'][i],y_pred[i],y_prob[i][y_pred[i]]])

sorted(out_need,key=lambda tup: tup[2],reverse=True)
sorted(out_avail,key=lambda tup: tup[2],reverse=True)
print (out_need)
#print (out_avail)

[[593751207719505921, 1, 1.0], [594833996405559297, 1, 1.0], [593553292501725185, 1, 1.0], [593996914347155456, 1, 1.0], [593502082092507137, 1, 1.0], [594789612918300672, 1, 1.0], [596936492573831168, 1, 1.0], [592748514003820545, 1, 1.0], [593302252493889536, 1, 1.0], [594506824910409730, 1, 1.0], [595254028050661376, 1, 1.0], [595075980185587713, 1, 1.0], [593685714337947648, 1, 1.0], [597345198285262848, 1, 1.0], [592747661092069377, 1, 1.0], [592934350880153600, 1, 1.0], [596917804688748544, 1, 1.0], [594880329493557248, 1, 1.0], [593382490699501569, 1, 1.0], [593093678752751617, 1, 1.0], [595134747132604417, 1, 1.0], [593660153938870272, 1, 1.0], [597315318751600640, 1, 1.0], [592996118386049026, 1, 1.0], [593091447424876546, 1, 1.0], [595963821497417729, 1, 1.0], [592751186152919042, 1, 1.0], [595043229940068352, 1, 1.0], [594329817794904065, 1, 1.0], [594495480458977280, 1, 1.0], [593218321409015809, 1, 1.0], [593286305779421184, 1, 1.0], [593698861673123840, 1, 1.0], [59309405

In [22]:
import csv
with open("LabelledTestDataset.csv","w+") as f:
    writer=csv.writer(f,delimiter=',')
    writer.writerow(['id','text','class','prob'])
    for i in range(0,len(y_pred)):
        writer.writerow([testset['id'][i],testset['text'][i],y_pred[i],y_prob[i][y_pred[i]]])
f.close()   

In [23]:
with open("outputPhase1.txt","w+") as f:
    for i in range(0,len(out_avail)):
        f.write("Nepal-Avail "+"Q0 "+str(out_avail[i][0])+" "+str(i+1)+" "+str(out_avail[i][2])+" iiests_IRMiDis_FIRE2017_1"+"\n")
    for i in range(0,len(out_need)):
        f.write("Nepal-Need "+"Q0 "+str(out_need[i][0])+" "+str(i+1)+" "+str(out_need[i][2])+" iiests_IRMiDis_FIRE2017_1"+"\n")
f.close()       